In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split

In [2]:
titanic = pd.read_csv("../dataset/titanic/train.csv")

In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [4]:
drop_cols = ['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked']

In [5]:
titanic = titanic.drop(drop_cols, axis=1)

In [12]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
dtypes: float64(1), int64(5)
memory usage: 41.8 KB


In [9]:
le = LabelEncoder()
titanic.Sex = le.fit_transform(titanic.Sex)

In [11]:
titanic.Age.fillna(np.mean(titanic.Age), inplace=True)

In [21]:
X = titanic.drop(["Survived"], axis=1)
y = titanic.Survived

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [29]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(10)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.7627118644067796


In [24]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.8135593220338984


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [33]:
def entropy(col):
    items, counts = np.unique(col.values, return_counts=True)
    size = col.values.shape[0]
    acc = 0
    for count in counts:
        pi = count/size
        acc += (pi * np.log2(pi))
    
    return -acc


In [35]:
vals = pd.Series([1, 1, 1, 0, 0, 0])
print(entropy(vals))

1.0


In [32]:
print(items, counts)

[0 1] [549 342]


In [41]:
def info_gain(X, Y, label):
    mean = np.mean(X[label])

    left_y = Y[X[label]<mean]
    right_y = Y[X[label]>=mean]
    
    s_total = Y.shape[0]
    s_left = left_y.shape[0]
    s_right = right_y.shape[0]
    
    if s_left == 0 or s_right == 0:
        return -10000
    
    return entropy(Y) - ((s_left/s_total)*entropy(left_y) + (s_right/s_total)*entropy(right_y))
    

In [45]:

for col in X_train.columns:
    print(col, info_gain(X_train, y_train, col))

Pclass 0.0693612979550764
Sex 0.20332074264151545
Age 0.0016815830685426025
SibSp 0.004524587061436547
Parch 0.014104142987097612


In [46]:
X_train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch'], dtype='object')

In [47]:
class Node:
    
    def __init__(self, label=None, value=None, result=None):
        self.label = label
        self.value = value
        self.result = result
        
        self.left = None
        self.right = None

In [74]:
class DecisionTree:
    
    def __init__(self):
        self.root = None
        
    def best_col(self, x_data, y_data):
        gains = []
        
        for col in x_data.columns:
            gains.append((info_gain(x_data, y_data, col), col))
                         
        gains = sorted(gains)
        
        return gains[-1][1]
        
        
    def generate(self, x_data, y_data, max_dep=5):
        
        if max_dep == 0:
            return Node(result=np.mean(y_data))
        
        best = self.best_col(x_data, y_data)
        
        mean = np.mean(x_data[best])

        left_x = x_data[x_data[best]<mean]
        right_x = x_data[x_data[best]>=mean]
        
        left_y = y_data[x_data[best]<mean]
        right_y = y_data[x_data[best]>=mean]
        
        if left_y.shape[0] == 0 or right_y.shape[0] == 0:
            return Node(result=np.mean(y_data))
        
        node = Node(label=best, value=mean)
        
        node.left = self.generate(left_x, left_y, max_dep-1)
        node.right = self.generate(right_x, right_y, max_dep-1)
        
        return node
        
    
    def display(self, node, indent=0):
        if node == None:
            return
        
        print("\t"*indent, node.label, node.value, node.result)
        
        self.display(node.left, indent+1)
        self.display(node.right, indent+1)
        
            
        

In [83]:
tree = DecisionTree()

In [86]:
tree.root = tree.generate(X_train, y_train, 4)

In [87]:
tree.display(tree.root)

 Sex 0.6543624161073825 None
	 Pclass 2.1941747572815533 None
		 SibSp 0.514018691588785 None
			 Age 32.97252371916508 None
				 None None 1.0
				 None None 0.9642857142857143
			 Age 29.644444444444446 None
				 None None 0.875
				 None None 1.0
		 SibSp 0.9797979797979798 None
			 Parch 0.5555555555555556 None
				 None None 0.5384615384615384
				 None None 0.6
			 Parch 1.1111111111111112 None
				 None None 0.4666666666666667
				 None None 0.13333333333333333
	 Pclass 2.4128205128205127 None
		 Parch 0.20915032679738563 None
			 Pclass 1.5 None
				 None None 0.3125
				 None None 0.109375
			 Age 29.06 None
				 None None 0.6153846153846154
				 None None 0.25
		 Age 27.647099776619505 None
			 Parch 0.3548387096774194 None
				 None None 0.17391304347826086
				 None None 0.2916666666666667
			 Parch 0.14583333333333334 None
				 None None 0.12121212121212122
				 None None 0.0
